In [5]:
import igl # Library to load meshes and perform operations on them
import meshplot as mp # Library to visualize meshes and point clouds
import vedo as vd # Library to visualize meshes and point clouds
import polyscope as ps # Library to visualize meshes
import numpy as np # Library to perform operations on matrices
import os # Library to perform operations on files and directories

# Importing the classes and functions from the geometry folder
from geometry.mesh import Mesh 
from geometry.utils import *

# Importing the classes and functions from the optimization folder
from optimization.Planarity import Planarity
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

# Importing the classes and functions from the visualization folder
vd.settings.default_backend = 'k3d'

# Directory path
dir_path = os.getcwd()

In [6]:
import igl # Library to load meshes and perform operations on them
import meshplot as mp # Library to visualize meshes and point clouds
import vedo as vd # Library to visualize meshes and point clouds
import polyscope as ps # Library to visualize meshes
import numpy as np # Library to perform operations on matrices
import os # Library to perform operations on files and directories

# Importing the classes and functions from the geometry folder
from geometry.mesh import Mesh 
from geometry.utils import *

# Importing the classes and functions from the optimization folder
from optimization.Planarity import Planarity
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

# Importing the classes and functions from the visualization folder
vd.settings.default_backend = 'k3d'

# Directory path
dir_path = os.getcwd()

ModuleNotFoundError: No module named 'geometry'

# Load Meshes

## Read Meshes

In [7]:
# You can use either igl or Mesh to load meshes

# igl can only load triangular meshes, it return a tuple (V, F)
v, f = igl.read_triangle_mesh(dir_path+"/models/Hall.obj")

# If you use the self implemented Mesh class, you can load any type of mesh
mesh = Mesh() # Create an empty mesh
mesh.read_obj_file(dir_path+"/models/Hall.obj") # Load the mesh from the obj file

# igl only return v and f. However the Mesh class has implemented a Half Edge data structure
# More information: https://jerryyin.info/geometry-processing-algorithms/half-edge/
# You can check the folder geometry/mesh.py to see how the half edge data structure is implemented

# To acces the vertices and faces of the mesh you can use the following commands
vertices = mesh.vertices
faces = mesh.faces()

print(f"igl vertices:\n {v[:5]},\n igl faces:\n {f[:5]}")

print(f"Mesh vertices:\n {vertices[:5]},\n Mesh faces:\n {faces[:5]}")

Mesh Data Structure: |V| = 3330, |F| = 6293, |E| = 9622
igl vertices:
 [[ 8.147727  8.511167  5.435742]
 [ 8.48664   7.833783  5.361411]
 [ 7.799988  8.175888  4.931837]
 [32.59761  20.012056  5.150359]
 [32.977852 20.232679  4.539225]],
 igl faces:
 [[ 1  0  2]
 [ 4  3  5]
 [ 7  6  8]
 [10  9 11]
 [13 12 14]]
Mesh vertices:
 [[ 8.147727  8.511167  5.435742]
 [ 8.48664   7.833783  5.361411]
 [ 7.799988  8.175888  4.931837]
 [32.59761  20.012056  5.150359]
 [32.977852 20.232679  4.539225]],
 Mesh faces:
 [[ 1  0  2]
 [ 4  3  5]
 [ 7  6  8]
 [10  9 11]
 [13 12 14]]


## Create Meshes

You can create meshes by defining its vertices and faces list. 

In [5]:
# Random vertices
v = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 1],
    ])

# Random faces
f = np.array([
    [0, 1, 2],
    [0, 2, 3],
    ])

# Create a mesh from vertices and faces
mesh = Mesh()
mesh.make_mesh(v, f)

print(f"Mesh vertices:\n {mesh.vertices},\n Mesh faces:\n {mesh.faces()}")

Mesh Data Structure: |V| = 4, |F| = 2, |E| = 5
Mesh vertices:
 [[0. 0. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [0. 1. 1.]],
 Mesh faces:
 [[0 1 2]
 [0 2 3]]


One of the advantages of the Halfedge data structure is that we can acces very easily to certain properties of the mesh. For example obtaining the faces neighbor to a vertex

In [11]:
# Neighbor faces to vertices
# Each row contain the neighbor faces index to the vertex with the same index
nf = mesh.vertex_ring_faces_list()

# Neighbor vertices to vertex 0 
print(nf[0])

[0, 1, -1]


The number -1 refers to a halfedge with no face or boundary halfedge. In general means that vertex 0 is a boundary vertex. 

# Visualization 

So far we have seen how to load meshes and create meshes but we haven't visualize them. Here I will show the alternatives for visualization either using **meshplot** or **vedo**. 

The adventage of **meshplot** is that is really easy to use and fast. Moreover, it is possible to visualize the evolution of an optimization process. However, it only work for triangular meshes and for ourporpuses is not always the case.

[**Vedo**](https://vedo.embl.es/docs/vedo.html) on the other hand allow to visualize any kind of polygonal meshes without restrictions, and also contain some mesh operations that can be helpfull from the geometric processing point of view. For example mesh intersection, boolean operations, group, etc. 

## Mesh plot

In [6]:
# Load mesh
# Random vertices
v = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 1],
    ])

# Random faces
f = np.array([
    [0, 1, 2],
    [0, 2, 3],
    ])

# Visualize mesh
mp.plot(v, f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.5,…

Polygonal shape problems. In the following example we will try to visualize a simple hexagon given by 6 vertices and only one face. 

In [13]:
# Create an hex mesh
# Hexagon center at the origin
v = np.array([
    [0, 0, 0], # 0
    [1, 0, 0], # 1
    [1.5, np.sqrt(3) / 2, 0], # 2
    [1, np.sqrt(3), 0], # 3
    [0, np.sqrt(3), 0], # 4
    [-0.5, np.sqrt(3) / 2, 0] # 5
])

    

# Faces
f = np.array([
    [0, 1, 2, 3, 4, 5]
    ])

# Visualize mesh
mp.plot(v, f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.86…

We can see that meshplot interpret the face as two separated triangles instead of drawing the correct shape of the hexagon. We can do some extra work to triangulate the hexagon but it could be tedius and a waste of time.

## Vedo

In [5]:
# Load mesh
# Random vertices
v = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 1],
    ])

# Random faces
f = np.array([
    [0, 1, 2],
    [0, 2, 3],
    ])

# Create mesh vedo
mesh = vd.Mesh([v, f], c="red", alpha=0.5)

# Visualize wireframe
edges = mesh.clone().wireframe().c("black").lw(0.1)

# Visualize mesh
vd.show(mesh, edges)


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

As you can see to use vedo we need to define more things to obtain a nice visualization of the mesh in comparison with meshplot. However, we can use it to visualize any polygonal mesh. 

In [7]:
# Create an hex mesh
# Hexagon center at the origin
v = np.array([
    [0, 0, 0], # 0
    [1, 0, 0], # 1
    [1.5, np.sqrt(3) / 2, 0], # 2
    [1, np.sqrt(3), 0], # 3
    [0, np.sqrt(3), 0], # 4
    [-0.5, np.sqrt(3) / 2, 0] # 5
])

    

# Faces
f = np.array([
    [0, 1, 2, 3, 4, 5]
    ])

# Visualize mesh vedo
mesh = vd.Mesh([v, f], c="red", alpha=0.5)

# Visualize wireframe
edges = mesh.clone().wireframe().c("black").lw(0.1)

# Visualize mesh
vd.show(mesh, edges)


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

As you can see vedo triangulate the shape automatically and shows the correct shape. But let it try in a more complicated example made using Mesh class. 

In [8]:
# Load mesh
mesh = Mesh()
mesh.read_obj_file(dir_path+"/models/catenoid_def_1.obj")

# Faces 
f = mesh.faces()
# Vertices
v = mesh.vertices

# Compute barycenter of the mesh
bar = v[f].mean(axis=1)

# Get dual topology of the mesh
dual = mesh.dual_top()


#Visualize mesh vedo
mesh = vd.Mesh([v, f], c="red", alpha=0.5)

#Vis dual mesh vedo
dual_mesh = vd.Mesh([bar, dual], c="blue", alpha=0.5)

dual_mesh.lw(2.5).lc('white')

#Visualize mesh
vd.show(mesh, dual_mesh, __doc__, axes=11, viewup="z")

Mesh Data Structure: |V| = 336, |F| = 601, |E| = 936


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

It visualize the correct mesh but not show edges. 

## Polyscope

Other alternative  for visualization is polyscope, which is so far the one that work the best but is used only for visualization. 

In [8]:
# Initialize polyscope
ps.init()


### Register a mesh
# `verts` is a Nx3 numpy array of vertex positions
# `faces` is a Fx3 array of indices, or a nested list
ps.register_surface_mesh("Mesh", v, f, smooth_shade=True)
#ps.register_surface_mesh("Dual", bar, dual, smooth_shade=True)

# Add a scalar function and a vector function defined on the mesh
# vertex_scalar is a length V numpy array of values
# face_vectors is an Fx3 array of vectors per face


# View the point cloud and mesh we just registered in the 3D UI
ps.show()


# Optimization (Framework)

## Planarity

Here we are going to implement a simple minimization problem. We are goint to consider a mesh with four quad faces, and we want the faces to be planar. 
The mesh $M = \{ V, \ F, \ E \}$, where $V$ is the set of vertices, $F$ the set of faces and $E$ the set of edges. The planarity condition requires that per each face $f = v_i v_j v_k v_l$ the four vertices are coplanar. There are many geometric ways to impose this but the most efficient and easy is to add an auxiliary variable $n_f$ that represent a normal vector per ach face and we optimize per each face the energy:
$$  || n_f\cdot (v_j - v_i) ||^2; \quad \ \ v_i, \ v_f \in E(f).$$

Let's start by defining the initial mesh with not planar quads.

In [30]:
# Vertices
v = np.array(
    [
        [ 0.01,  0.01,  0.8],
        [ 1.01,  0.03,  0.01],
        [-1.02,  0.02,  0.01],
        [ 0.01,  1.1, -0.2],
        [ 0.01, -1.3, -0.3],
        [-1.02,  1.01,  0.1],
        [ 1.01,  1.02,  0.2],
        [-1.04, -1.03, -0.3],
        [ 1.05, -1.04,  0.1],
    ]
   )
# Faces
fcs = np.array([[0, 1, 6, 3], 
                [2, 0, 3, 5], 
                [7, 4, 0, 2], 
                [4, 8, 1, 0]])



The full energy that we want to minimize is,
$$ E_{planarity} = \sum_{f\in F} \sum_{v_i, v_j \in E(f)} || n_f \cdot (v_j - v_i) ||^2 + \sum_{f \in F} || n_f \cdot n_f - 1 ||^2$$

The method that we are going to use the called [Levenberg-Marquart](https://en.wikipedia.org/wiki/Levenberg%E2%80%93Marquardt_algorithm). For simplicity I am going to describe the main things we need to optimize the energy. 

The main idea is that our energy is of the form:
$$ E = \sum_{i}^n || \phi_i(X) ||^2, $$
where $X\in \mathbb{R}^m$  is a vector of variables and $\phi: \mathbb{R}^m \to \mathbb{R}$. Then, what the LM method does is to solve iteratively a linear system that will guide us to an optimal solution,
$$ (J^T J + \lambda \mathbb{I}) \delta_x =  - J^T\ r,$$
where
$$ J_{ij} = \frac{ \partial \phi_i{X} }{\partial x_j},$$
$$ r_i = \phi_i(X).$$
$\lambda$ is a parameter avoid the non solvability in general is a small value that in practice is computed as the maximum diagonal entry of $J^T J$ times $10^-6$.

## $J$ Computation

Let's see how we can compute $J$ for our example. We can see that $J$ will be a matrix where the rows are equivalent to each constraint or function $\phi_i$ and each column correspond to the derivatives of our variables. In our problem our variables are the vertices $V$ and the auxiliar normals $n_f$. Moreover, we have one constriant per each edge in each face. We can even rewrite the energy as,
$$ E_{planarity} = \sum_{f\in F} \sum_{i = 0}^3 || n_f \cdot (v_i - v_{i+1}) ||^2 + \sum_{f \in F} || n_f \cdot n_f - 1 ||^2$$
where the subscripts of the vertices are taken as $\mod 4$. This means that we have in total $4|F|$ per each edge in a quad plus $|F|$ in the second sum related to the normalization of the normal vectors as constaints, and $3|V|+3|F|$ variables. We consider $3|V|$ because we have three coordinates per each vertex and three coodinate per each normal vector $n_f$.

In [6]:
V = len(v) # Number of vertices
F = len(fcs) # Number of faces

# Set dictionary of variables indices in this case vetices and normals of faces
var_idx = {
    "v"  : np.arange(0, 3*V),
    "nf" : np.arange(3*V, 3*V + 3*F)
}

# Init X
X = np.zeros(3*V + 3*F) 

Now we need to create a class constraint that take as input the information of the mesh $v$ and $f$ and put the corresponding values in the matrix $J$. Let's define the structure of $J$ as the first $3|V|$ columns related to the vertices derivatives and $F$ related to the normal derivatives. Fixing $i$ and $f$ we can see that the 
$$\partial_{x_i} ( n_f \cdot(x_i - x_{i+1}) ) = n_f,$$
$$\partial_{x_{i+1}} ( n_f \cdot(x_i - x_{i+1}) ) = - n_f,$$
$$\partial_{n_{f}} ( n_f \cdot(x_i - x_{i+1}) ) = (x_i - x_{i+1}).$$
$$\partial_{n_{f}} ( n_f \cdot n_f - 1) ) = n_f$$

Let's remember that $f$ can be the index of the face in the list of faces *fcs* and *i* means the vertex indices in *fcs[f]*.

**Remark:** In the previus part we define the derivative with respect to $\partial_{x_i}$ but $x_i$ is a vector which means that what we need to fill in $J$ is the derivative with respect to each coodinate, i.e., $(\partial_{x_i})_x ( n_f \cdot(x_i - x_{i+1}) ) = (n_f)_x $

Let's define our class planarity. This in practice should be done in a separated file. We are goint to inherit from a super class called constraint that can be found in ```\optimization\constraint.py``` bellow you will se a sample of how to create a constraint class four our problem.

In [31]:
import numpy as np
import geometry as geo
from optimization.constraint import Constraint
from optimization.Optimizer import Optimizer

class Planarity(Constraint):

    def __init__(self) -> None:
        """
        Here you need to define all the variables that you will use in the constraint
        that are not stored in X.

        For example if you need to use the normals you store them in a variable like this:
        self.normals = None

        The idea is that you initialize all the variables that you will use in the constraint
        in the initialize_constraint function and then you can use them when you are computing 
        J and r.
        """ 
        super().__init__()
        # In this case we don't need any extra variables
        

    def initialize_constraint(self, X, var_idx, F) -> np.array:
        """ 
        Here you need to initialize all the variables that you will use in the constraint,
        define the constraint indices as a dictionary similar to the variable indices, and
        the number of constraints and variables.
        

        In this case we are computing the initial normals of the faces and storing them in
        the X vector.
        """ 
        

        # Set variables indices
        self.var_idx = var_idx

        # number of faces
        nF = F.shape[0]

        
        # Constraint indices
        # For the planarity constraint we have 4 constraints per each face, corresponding to each edge
        # We will define the indices of the constraints in a dictionary per ach edge as
        # ed1 = (v1 - v0)
        # ed2 = (v2 - v1)
        # ed3 = (v3 - v2)
        # ed4 = (v0 - v3)
        # For the energy ||n.n -1 ||^2 we will define the index as unit_norm
        self.const_idx = {
            "ed1" : np.arange(   0,   nF),
            "ed2" : np.arange(  nF, 2*nF),
            "ed3" : np.arange(2*nF, 3*nF),
            "ed4" : np.arange(3*nF, 4*nF),
            "unit_norm" : np.arange(4*nF, 5*nF)
        }
        
        # Uncurry is a function that extract the variables from X and store them in a variable
        # uncurry made use of the dictionary of variables indices that we define before
        v = self.uncurry_X(X, "v")
        v = v.reshape(-1, 3)
        
        vi, vj, vk, vl = v[F[:,0]], v[F[:,1]], v[F[:,2]], v[F[:,3]]

        # Compute diagonals of the faces
        diag1 = vk - vi 
        diag2 = vl - vj

        self.const = 3*len(v) + 4*len(F) # Number of constraints
        self.var = len(X)

        # # Compute normals
        X[var_idx["nf"]] = np.cross(diag1, diag2).flatten()
    

    
    def compute(self, X, F) -> None:
        """
        The most important function to create in the class is compute. 
        This is where you are goint to define the matrix J and r. 
        
        For this you will use the functions add_derivatives and set_r. 

        The structure of compute is (X, *args), X variables and *args any other argument needed for the
        computation.
        """ 
        
        # Get variables from X
        v, nf = self.uncurry_X(X, "v", "nf")

        # Remember that X has the variables in a vector, so we need to reshape them if needed
        v = v.reshape(-1, 3)
        nf = nf.reshape(-1, 3)

        # Get var and constraint indices
        var_idx = self.var_idx
        c_idx = self.const_idx

        # Get indices of vertices per each face
        v0_idx = var_idx["v"][3*F[:,0]].repeat(3) + np.tile(np.arange(3), F.shape[0])
        v1_idx = var_idx["v"][3*F[:,1]].repeat(3) + np.tile(np.arange(3), F.shape[0])
        v2_idx = var_idx["v"][3*F[:,2]].repeat(3) + np.tile(np.arange(3), F.shape[0])
        v3_idx = var_idx["v"][3*F[:,3]].repeat(3) + np.tile(np.arange(3), F.shape[0])
        nf_idx = var_idx["nf"]
        
        # Planarity
        # Constraint  n . (vi - vj) per each edge (vi, vj) in F, so we have 4 constraints per each face
        # Our variarles are the vertices of the mesh, so we have 3n variables plus the auxiliary variable
        # n the normals of the faces which result in 3*n + 3*F variables

        v0, v1, v2, v3 = v[F[:,0]], v[F[:,1]], v[F[:,2]], v[F[:,3]]
        

        # Energy: edg1 =  ||nf.(v1 - v0)||^2 
        # d v0 = - nf
        self.add_derivatives(c_idx["ed1"].repeat(3), v0_idx, - nf.flatten())
        # d v1 =   nf 
        self.add_derivatives(c_idx["ed1"].repeat(3), v1_idx,   nf.flatten())
        # d nf = (v1 - v0)
        self.add_derivatives(c_idx["ed1"].repeat(3), nf_idx, (v1 - v0).flatten())
        # r = nf.(v1 - v0)
        self.set_r(c_idx["ed1"], vec_dot(nf,(v1 - v0)) )

        # Energy: edg2 =  ||nf.(v2 - v1)||^2
        # d v1 = - nf
        self.add_derivatives(c_idx["ed2"].repeat(3), v1_idx, - nf.flatten())
        # d v2 =   nf
        self.add_derivatives(c_idx["ed2"].repeat(3), v2_idx,   nf.flatten())
        # d nf = (v2 - v1)
        self.add_derivatives(c_idx["ed2"].repeat(3), nf_idx, (v2 - v1).flatten())
        # r = nf.(v2 - v1)
        self.set_r(c_idx["ed2"], vec_dot(nf, v2 - v1))

        # Energy: edg3 =  ||nf.(v3 - v2)||^2
        # d v2 = - nf
        self.add_derivatives(c_idx["ed3"].repeat(3), v2_idx, - nf.flatten())
        # d v3 =   nf
        self.add_derivatives(c_idx["ed3"].repeat(3), v3_idx,   nf.flatten())
        # d nf = (v3 - v2)
        self.add_derivatives(c_idx["ed3"].repeat(3), nf_idx, (v3 - v2).flatten())
        # r = nf.(v3 - v2)
        self.set_r(c_idx["ed3"], vec_dot(nf, v3 - v2))

        # Energy: edg4 =  ||nf.(v0 - v3)||^2
        # d v3 = - nf
        self.add_derivatives(c_idx["ed4"].repeat(3), v3_idx, - nf.flatten())
        # d v0 =   nf
        self.add_derivatives(c_idx["ed4"].repeat(3), v0_idx,   nf.flatten())
        # d nf = (v0 - v3)
        self.add_derivatives(c_idx["ed4"].repeat(3), nf_idx, (v0 - v3).flatten())
        # r = nf.(v0 - v3)
        self.set_r(c_idx["ed4"], vec_dot(nf, v0 - v3))

        # Energy: unit_norm =  ||nf.nf - 1||^2
        # d nf = nf
        self.add_derivatives(c_idx["unit_norm"].repeat(3), nf_idx, nf.flatten())
        # r = nf.nf - 1
        self.set_r(c_idx["unit_norm"], vec_dot(nf, nf) - np.ones(len(nf)))



Once, define the class and the computation of $J$ and $r$ they are solved using $$ (J^T J + \lambda \mathbb{I}) \delta_x =  - J^T\ r,$$
Now, let's define the initial X and let setup our optimization. 


In [32]:
# Set initial variables
X[var_idx["v"]] = v.flatten()

# Initialize planarity class
planarity = Planarity()
planarity.initialize_constraint(X, var_idx, fcs)

# Initialize optimizer
opt = Optimizer()
opt.initialize_optimizer(X, "LM", 0.8)

#iterations
it = 50
for _ in range(it):

    # Add constraint
    opt.add_constraint(planarity, fcs) 
    
    # Optimize
    opt.optimize()

X = opt.get_variables()

 E 1: 171.85628720367802 	 4.50916127565564
 E 2: 2.533933416268845 	 0.6859718071000227
 E 3: 0.19330872392643192 	 0.19516909212604452
 E 4: 0.015739690654741137 	 0.062276305429137424
 E 5: 0.0014052991581612667 	 0.021910163721113655
 E 6: 0.00014539172221919916 	 0.008527799299421842
 E 7: 1.8101203658459283e-05 	 0.003525665351478177
 E 8: 2.729501033647266e-06 	 0.0015108363069859293
 E 9: 4.665901060262871e-07 	 0.0006570846995316694
 E 10: 8.557830875309253e-08 	 0.00028780200643583866
 E 11: 1.6187172954724312e-08 	 0.0001263851985821652
 E 12: 1.4277504664337224e-09 	 3.743156898097898e-05
 E 13: 1.2624083551263042e-10 	 1.1088037750249739e-05
 E 14: 1.120360770299211e-11 	 3.285591221464046e-06
 E 15: 9.993004272603585e-13 	 9.740061568259668e-07
Best iteration: 15	 Best energy: 9.993004272603585e-13


In [34]:
# Visualization

# Get variables from X
nv, _ = planarity.uncurry_X(X, "v", "nf")

# Initialize polyscope
ps.init()

### Register a mesh
ps.register_surface_mesh("Init_Mesh", v, fcs, smooth_shade=True)
ps.register_surface_mesh("Opt_Mesh", nv.reshape(-1, 3), fcs, smooth_shade=True)

# View the point cloud and mesh we just registered in the 3D UI
ps.show()

ps.init()


### Register a mesh
ps.register_surface_mesh("Init_Mesh", v, fcs, smooth_shade=True)
ps.register_surface_mesh("final", vertices, fcs, smooth_shade=True)

# View the point cloud and mesh we just registered in the 3D UI
ps.show()